In [2]:
# Use the alarm model to generate data from it
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling

alarm_model = get_example_model("alarm")
samples = BayesianModelSampling(alarm_model).forward_sample(size=int(1e5))
samples.head()

Generating for node: BP: 100%|██████████| 37/37 [00:03<00:00, 12.25it/s]          


,HISTORY,CVP,PCWP,HYPOVOLEMIA,LVEDVOLUME,LVFAILURE,STROKEVOLUME,ERRLOWOUTPUT,HRBP,HREKG,ERRCAUTER,HRSAT,INSUFFANESTH,ANAPHYLAXIS,TPR,EXPCO2,KINKEDTUBE,MINVOL,FIO2,PVSAT,SAO2,PAP,PULMEMBOLUS,SHUNT,INTUBATION,PRESS,DISCONNECT,MINVOLSET,VENTMACH,VENTTUBE,VENTLUNG,VENTALV,ARTCO2,CATECHOL,HR,CO,BP
0,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,FALSE,HIGH,FALSE,FALSE,HIGH,LOW,FALSE,ZERO,NORMAL,LOW,LOW,NORMAL,FALSE,NORMAL,NORMAL,NORMAL,FALSE,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,NORMAL
1,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,LOW,FALSE,HIGH,HIGH,FALSE,HIGH,FALSE,FALSE,HIGH,LOW,FALSE,ZERO,LOW,LOW,LOW,NORMAL,FALSE,NORMAL,NORMAL,HIGH,FALSE,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,LOW,NORMAL
2,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,LOW,HIGH,TRUE,LOW,FALSE,FALSE,HIGH,LOW,FALSE,HIGH,NORMAL,HIGH,HIGH,NORMAL,FALSE,NORMAL,NORMAL,HIGH,TRUE,NORMAL,NORMAL,ZERO,LOW,HIGH,LOW,NORMAL,NORMAL,NORMAL,NORMAL
3,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,NORMAL,TRUE,NORMAL,TRUE,FALSE,HIGH,LOW,FALSE,ZERO,NORMAL,LOW,LOW,LOW,FALSE,NORMAL,NORMAL,LOW,FALSE,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,HIGH
4,FALSE,HIGH,HIGH,FALSE,HIGH,FALSE,NORMAL,TRUE,LOW,NORMAL,TRUE,NORMAL,FALSE,FALSE,HIGH,LOW,FALSE,NORMAL,NORMAL,LOW,LOW,LOW,FALSE,HIGH,ONESIDED,HIGH,FALSE,NORMAL,NORMAL,LOW,ZERO,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,HIGH


In [3]:
# Defining the Bayesian Model Structure
from pgmpy.models import BayesianNetwork

model_struct = BayesianNetwork(ebunch=alarm_model.edges())
model_struct.nodes()


NodeView(('HYPOVOLEMIA', 'LVEDVOLUME', 'STROKEVOLUME', 'CVP', 'PCWP', 'LVFAILURE', 'HISTORY', 'CO', 'ERRLOWOUTPUT', 'HRBP', 'ERRCAUTER', 'HREKG', 'HRSAT', 'INSUFFANESTH', 'CATECHOL', 'ANAPHYLAXIS', 'TPR', 'BP', 'KINKEDTUBE', 'PRESS', 'VENTLUNG', 'FIO2', 'PVSAT', 'SAO2', 'PULMEMBOLUS', 'PAP', 'SHUNT', 'INTUBATION', 'MINVOL', 'VENTALV', 'DISCONNECT', 'VENTTUBE', 'MINVOLSET', 'VENTMACH', 'EXPCO2', 'ARTCO2', 'HR'))

In [4]:
# Fitting the model using Maximum Likelihood Estimator
from pgmpy.estimators import MaximumLikelihoodEstimator

mle = MaximumLikelihoodEstimator(model=model_struct, data=samples)

# Estimating the CPD for a single node
print(mle.estimate_cpd(node="FIO2"))
print(mle.estimate_cpd(node="CVP"))

# Estimating CPDs for all the nodes in the model
mle.get_parameters()[:10] # Show the first 10 CPDs in the output

+--------------+--------+
| FIO2(LOW)    | 0.0499 |
+--------------+--------+
| FIO2(NORMAL) | 0.9501 |
+--------------+--------+
+-------------+-----+----------------------+
| LVEDVOLUME  | ... | LVEDVOLUME(NORMAL)   |
+-------------+-----+----------------------+
| CVP(HIGH)   | ... | 0.009920606639394501 |
+-------------+-----+----------------------+
| CVP(LOW)    | ... | 0.04011003891272432  |
+-------------+-----+----------------------+
| CVP(NORMAL) | ... | 0.9499693544478812   |
+-------------+-----+----------------------+


[<TabularCPD representing P(HYPOVOLEMIA:2) at 0x7f7055fc16a0>,
 <TabularCPD representing P(LVEDVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f7054d6c460>,
 <TabularCPD representing P(STROKEVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f7056078190>,
 <TabularCPD representing P(CVP:3 | LVEDVOLUME:3) at 0x7f7056034520>,
 <TabularCPD representing P(PCWP:3 | LVEDVOLUME:3) at 0x7f7055fc1df0>,
 <TabularCPD representing P(LVFAILURE:2) at 0x7f7055fa5dc0>,
 <TabularCPD representing P(HISTORY:2 | LVFAILURE:2) at 0x7f7056050ac0>,
 <TabularCPD representing P(CO:3 | HR:3, STROKEVOLUME:3) at 0x7f7055e38ac0>,
 <TabularCPD representing P(ERRLOWOUTPUT:2) at 0x7f70560348b0>,
 <TabularCPD representing P(HRBP:3 | ERRLOWOUTPUT:2, HR:3) at 0x7f7056078160>]

In [5]:
# Verifying that the learned parameters are almost equal
import numpy as np

np.allclose(
    alarm_model.get_cpds("FIO2").values, mle.estimate_cpd("FIO2").values, atol=0.01    
)

True

In [6]:
# Fitting using the Bayesian Estimator
from pgmpy.estimators import BayesianEstimator

best = BayesianEstimator(model=model_struct, data=samples)

print(best.estimate_cpd(node="FIO2", prior_type="BDeu", equivalent_sample_size=1000))

# Uniform psuedo count for each state. Can also accept an array of the size of CPD.
print(best.estimate_cpd(node="CVP", prior_type="dirichlet", pseudo_counts=100))

# Learning CPDs for all the nodes in the model. For learning all parameters with BDeU prior, a dict of
# pseudo_counts need to be provided
best.get_parameters(prior_type="BDeu", equivalent_sample_size=1000)[:10]

+--------------+-----------+
| FIO2(LOW)    | 0.0543564 |
+--------------+-----------+
| FIO2(NORMAL) | 0.945644  |
+--------------+-----------+
+-------------+-----+----------------------+
| LVEDVOLUME  | ... | LVEDVOLUME(NORMAL)   |
+-------------+-----+----------------------+
| CVP(HIGH)   | ... | 0.011297670919851824 |
+-------------+-----+----------------------+
| CVP(LOW)    | ... | 0.0413585591211661   |
+-------------+-----+----------------------+
| CVP(NORMAL) | ... | 0.9473437699589821   |
+-------------+-----+----------------------+


[<TabularCPD representing P(HYPOVOLEMIA:2) at 0x7f7054522280>,
 <TabularCPD representing P(LVEDVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f7055e38e80>,
 <TabularCPD representing P(STROKEVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f705450ca60>,
 <TabularCPD representing P(CVP:3 | LVEDVOLUME:3) at 0x7f7054d879a0>,
 <TabularCPD representing P(PCWP:3 | LVEDVOLUME:3) at 0x7f7054d87df0>,
 <TabularCPD representing P(LVFAILURE:2) at 0x7f70560781f0>,
 <TabularCPD representing P(HISTORY:2 | LVFAILURE:2) at 0x7f7054d87c70>,
 <TabularCPD representing P(CO:3 | HR:3, STROKEVOLUME:3) at 0x7f705450cb20>,
 <TabularCPD representing P(ERRLOWOUTPUT:2) at 0x7f7056078a30>,
 <TabularCPD representing P(HRBP:3 | ERRLOWOUTPUT:2, HR:3) at 0x7f7055e38730>]